In [ ]:
from __future__ import division
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%precision 4
plt.style.use('ggplot')

In [ ]:
np.random.seed(1234)
import pystan
import scipy.stats as stats
import arviz as az

In [ ]:
import warnings
warnings.simplefilter('ignore', UserWarning)

PyStan
====

Install `PyStan`  with
```
pip install pystan
```

The nice thing about `PyMC` is that everything is in Python. With `PyStan`, however, you need to use a domain specific language based on C++ syntax to specify the model and the data, which is less flexible and more work. However, in exchange you get an extremely powerful HMC package (only does HMC) that can be used in R and Python.

### Useful links

- [Paper describing Stan](http://www.stat.columbia.edu/~gelman/research/unpublished/stan-resubmit-JSS1293.pdf)
- [Stan Examples and Reference Manual](https://github.com/stan-dev/example-models/wiki)
- [PyStan docs](http://pystan.readthedocs.org/en/latest/)
- [PyStan GitHub page](https://github.com/stan-dev/pystan)

In [ ]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

### Coin toss

We'll repeat the example of determining the bias of a coin from observed coin tosses. The likelihood is binomial, and  we use a beta prior.

In [ ]:
%%capture

coin_code = """
data {
    int<lower=0> n; // number of tosses
    int<lower=0> y; // number of heads
}
transformed data {}
parameters {
    real<lower=0, upper=1> p;
}
transformed parameters {}
model {
    p ~ beta(2, 2);
    y ~ binomial(n, p);
}
generated quantities {}
"""

coin_dat = {
             'n': 100,
             'y': 61,
            }

sm = pystan.StanModel(model_code=coin_code)

In [ ]:
fit = sm.sampling(data=coin_dat, iter=1000, chains=1)

#### Loading from a file

The string in coin_code can also be in a file - say `coin_code.stan` - then we can use it like so

```python
fit = pystan.stan(file='coin_code.stan', data=coin_dat, iter=1000, chains=1)
```

In [ ]:
print(fit)

In [ ]:
coin_dict = fit.extract()
coin_dict.keys() 
# lp_ is the log posterior

In [ ]:
fit.plot('p');
plt.tight_layout()

In [ ]:
az.plot_trace(fit)
pass

### Estimating mean and standard deviation of normal distribution

$$
X \sim \mathcal{N}(\mu, \sigma^2)
$$

In [ ]:
%%capture

norm_code = """
data {
    int<lower=0> n; 
    real y[n]; 
}
transformed data {}
parameters {
    real<lower=0, upper=100> mu;
    real<lower=0, upper=10> sigma;
}
transformed parameters {}
model {
    y ~ normal(mu, sigma);
}
generated quantities {}
"""

norm_dat = {
             'n': 100,
             'y': np.random.normal(10, 2, 100),
            }

sm = pystan.StanModel(model_code=norm_code)

In [ ]:
fit = sm.sampling(data=norm_dat, iter=1000, chains=1)

In [ ]:
fit

In [ ]:
trace = fit.extract()

In [ ]:
az.plot_density(trace)
pass

#### Optimization (finding MAP)

In [ ]:
op = sm.optimizing(data=norm_dat)
op

#### Reusing models

In [ ]:
new_dat = {
             'n': 100,
             'y': np.random.normal(5, 1, 100),
            }

In [ ]:
fit2 = sm.sampling(data=new_dat, chains=1)

In [ ]:
fit2

#### Saving compiled models

We can also compile Stan models and save them to file, so as to reload them for later use without needing to recompile.

In [ ]:
def save(obj, filename):
    """Save compiled models for reuse."""
    import pickle
    with open(filename, 'wb') as f:
        pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)

def load(filename):
    """Reload compiled models for reuse."""
    import pickle
    return pickle.load(open(filename, 'rb'))

In [ ]:
%%capture

model = pystan.StanModel(model_code=norm_code)
save(model, 'norm_model.pic')

In [ ]:
new_model = load('norm_model.pic')
fit4 = new_model.sampling(new_dat, chains=1)
fit4

### Estimating parameters of a linear regression model

We will show how to estimate regression parameters using a simple linear model

$$
y \sim ax + b
$$

We can restate the linear model $$y = ax + b + \epsilon$$ as sampling from a probability distribution

$$
y \sim \mathcal{N}(ax + b, \sigma^2)
$$

We will assume the following priors

$$
a \sim \mathcal{N}(0, 100) \\
b \sim \mathcal{N}(0, 100) \\
\sigma \sim \mathcal{U}(0, 20)
$$

In [ ]:
%%capture

lin_reg_code = """
data {
    int<lower=0> n; 
    real x[n];
    real y[n]; 
}
transformed data {}
parameters {
    real a;
    real b;
    real sigma;
}
transformed parameters {
    real mu[n];
    for (i in 1:n) {
        mu[i] <- a*x[i] + b;
        }
}
model {
    sigma ~ uniform(0, 20);
    y ~ normal(mu, sigma);
}
generated quantities {}
"""

n = 11
_a = 6
_b = 2
x = np.linspace(0, 1, n)
y = _a*x + _b + np.random.randn(n)

lin_reg_dat = {
             'n': n,
             'x': x,
             'y': y
            }

sm = pystan.StanModel(model_code=lin_reg_code)

In [ ]:
fit = sm.sampling(data=lin_reg_dat, iter=1000, chains=1)

In [ ]:
az.plot_trace(fit, ['a', 'b'])
pass

### Simple Logistic model

We have observations of height and weight and want to use a logistic model to guess the sex.

In [ ]:
# observed data
df = pd.read_csv('data/HtWt.csv')
df.head()

In [ ]:
%%capture

log_reg_code = """
data {
    int<lower=0> n; 
    int male[n];
    real weight[n];
    real height[n];
}
transformed data {}
parameters {
    real a;
    real b;
    real c;
}
transformed parameters {}
model {
    a ~ normal(0, 10);
    b ~ normal(0, 10);
    c ~ normal(0, 10);
    for(i in 1:n) {
        male[i] ~ bernoulli(inv_logit(a*weight[i] + b*height[i] + c));
  }
}
generated quantities {}
"""

log_reg_dat = {
             'n': len(df),
             'male': df.male,
             'height': df.height,
             'weight': df.weight
            }

sm = pystan.StanModel(model_code=log_reg_code)

In [ ]:
fit = sm.sampling(data=log_reg_dat, iter=2000, chains=1)

In [ ]:
df_trace = pd.DataFrame(fit.extract(['c', 'b', 'a']))
pd.plotting.scatter_matrix(df_trace[:], diagonal='kde');